In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from feature_extraction import Blob, Words
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from time import time
from pprint import pprint
from grab_posts import grab_posts

In [2]:
data = pd.read_csv('processed_datascience.csv.bz2')
#data = pd.read_csv('processed_python.csv.bz2', encoding='ISO-8859-1', compression='bz2')
#data = grab_posts(sub='datascience', number=30000)
data.drop_duplicates('title', inplace=True)

In [3]:
data['gtavg'] = data['ups'] >= data['ups'].mean()
data['gt5'] = data['ups'] > 5

train_X, test_X, train_y, test_y = train_test_split(data.title, 
                                                    data.gt5, 
                                                    test_size=0.20,
                                                    random_state=42)

In [5]:
print(data.ups.mean())
print(data[data['gt5']==False].shape)
print(data[data['gt5']==True].shape)

9.86234230211
(6732, 7)
(2225, 7)


In [63]:
pipeline = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[
                      
            ('words', Words()),
                      
            ('title', Pipeline([
                ('tfidf', TfidfVectorizer(ngram_range=(1,3),
                                          sublinear_tf=True,
                                          stop_words='english')),
                ('svd', TruncatedSVD(n_components=120)),
                ('normalize', MinMaxScaler(copy=False)),
                ('selector', SelectPercentile(f_classif,
                                              percentile=10))
            ])),
            
            ('blob', Pipeline([
                ('all', Blob()),
                ('minmax', MinMaxScaler()),
            ])),
            
            ])),
    ('clf', ExtraTreesClassifier()),
        ])

In [55]:
# ExtraTreesClassifier(n_estimators=120, max_depth=5, max_features='sqrt', min_samples_leaf=5, min_samples_split=10)
# RandomForestClassifier(n_estimators=1200, max_depth=5, max_features='sqrt', min_samples_leaf=5, min_samples_split=10)

In [6]:
pipeline.fit(train_X, train_y)

y = pipeline.predict(test_X)
accuracy_score(y_pred=y, y_true=test_y)

NameError: name 'pipeline' is not defined

In [57]:
cross_val_score(pipeline, train_X, train_y, cv=5)

array([ 0.57880056,  0.58507671,  0.60990928,  0.58659218,  0.60824022])

In [49]:

pipeline.predict(pd.Series(["I automated youtube content creation with python! What projects have you used python for lately?"]))[0]

False

In [10]:

'''
parameters = {
    'clf_n_estimators':(120,300,500,800,1200),
    'clf__max_depth':(5,8,15,25,30, None),
    'clf__min_samples_split':(2,5,10,15,100),
    'clf__min_samples_leaf':(1,2,5,10),
    'clf__max_features':('log2','sqrt',None),
}

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
grid_search.fit(train_X, train_y)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))


y = grid_search.predict(test_X)

accuracy_score(y_pred=y, y_true=test_y)
'''

'\nparameters = {\n    \'clf_n_estimators\':(120,300,500,800,1200),\n    \'clf__max_depth\':(5,8,15,25,30, None),\n    \'clf__min_samples_split\':(2,5,10,15,100),\n    \'clf__min_samples_leaf\':(1,2,5,10),\n    \'clf__max_features\':(\'log2\',\'sqrt\',None),\n}\n\ngrid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)\n\nprint("Performing grid search...")\nprint("pipeline:", [name for name, _ in pipeline.steps])\nprint("parameters:")\npprint(parameters)\nt0 = time()\ngrid_search.fit(train_X, train_y)\nprint("done in %0.3fs" % (time() - t0))\nprint()\n\nprint("Best score: %0.3f" % grid_search.best_score_)\nprint("Best parameters set:")\nbest_parameters = grid_search.best_estimator_.get_params()\nfor param_name in sorted(parameters.keys()):\n    print("\t%s: %r" % (param_name, best_parameters[param_name]))\n\n\ny = grid_search.predict(test_X)\n\naccuracy_score(y_pred=y, y_true=test_y)\n'